In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
sys.path.append(os.path.dirname(os.path.abspath('')))
from auto_ab.stratification.params import SplitBuilderParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2
from auto_ab.stratification.split_builder import StratificationSplitBuilder

In [2]:
from auto_ab.auto_ab.params import DataParams, SimulationParams, HypothesisParams, ResultParams, SplitterParams

# Кейс №6. Выделение групп

In [ ]:
df = pd.read_csv('./data/ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [ ]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 20000,
        'target1': 30000
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [ ]:
split_builder = StratificationSplitBuilder(df, split_builder_params)

In [6]:
split_2 = split_builder.collect()

INFO:auto_ab.stratification.split_builder:Calculate stratas for guest table
INFO:auto_ab.stratification.split_builder:156765 outliers found


Conversion A: 0.5000
Conversion B: 0.5000
Conversion A: 0.5000
Conversion B: 0.5000
Conversion A: 0.5000
Conversion B: 0.5000
